In [43]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from fuzzywuzzy import process, fuzz

In [44]:
from counting.szp_funcs import to_double, matcher, load_jobs, load_pay_types, months_year, print_df

In [45]:
job_106 = load_jobs()
pay_types = load_pay_types()

In [46]:
job_106.columns

Index(['pre_job', 'job_106'], dtype='object')

In [47]:
pay_types.columns

Index(['type', 'name'], dtype='object')

In [48]:
look_up_jobs = job_106['pre_job'].unique().tolist()
look_up_pay = pay_types['name'].unique().tolist()

In [72]:
dir_names = []#['C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Июнь 2022']
#months = months_year[:5]
path_to_dirt = 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\'

In [73]:
import os

for file in os.listdir(path_to_dirt):
    d = os.path.join(path_to_dirt, file)
    if os.path.isdir(d):
        dir_names.append(d)

In [74]:
dir_names

['C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Апрель 2022',
 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Июнь 2022',
 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Май 2022',
 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Март 2022',
 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Февраль 2022',
 'C:\\Users\\PetukhovMD\\Desktop\\2022_new\\Январь 2022']

In [75]:
months = ['apr', 'jun', 'may', 'mar', 'feb', 'jan']
post_path = 'C:\\Users\\PetukhovMD\\Desktop\\szp_2022\\'

In [76]:
nach_name = 'ДОНМ_Начисления работников_V2.xlsx'
pay_name = 'ДОНМ_Начисления работников по категориямv2.xlsx'
kadr_name = 'ДОНМ_Кадровые_паспортные данные v2.xlsx'

In [77]:
def give_day(row):
    if pd.isna(row['wday']) | pd.isna(row['nday']):
        return 0
    if row['wday'] >= row['nday']:
        return 1
    return 0

In [78]:
for i in range(len(dir_names)):
    nach = pd.read_excel(dir_names[i] + '\\' + nach_name)
    clean_nach = nach.iloc[:, [2, 4, 8, 10, 11, 12, 13, 14, 16, 19, 21, 22, 23]]
    clean_nach.columns = ['inn', 'snils', 'job_dirty', 'type', 'stv', 'status', 'sum_all', 'sum_curr', 'sum_prev', 'status_pref', 'wday', 'nday', 'spec']
    clean_nach['stv'] = clean_nach.apply(lambda row: to_double(row, 'stv'), axis=1)
    clean_nach = clean_nach.fillna({'sum_curr': 0, 'sum_prev': 0})
    clean_nach['sum'] = clean_nach.sum_curr + clean_nach.sum_prev
    clean_nach['day'] = clean_nach.apply(lambda row: give_day(row), axis=1)
    cleaning_jobs = pd.DataFrame(clean_nach['job_dirty'].unique().tolist()).rename(columns={0: 'job_dirty'})
    cleaning_jobs['pre_job'] = cleaning_jobs.apply(lambda row: matcher(row, look_up_jobs, 'job_dirty'), axis=1)
    cleaning_jobs = pd.merge(cleaning_jobs, job_106, how='left').drop(columns=['pre_job']).rename(columns={'job_106': 'job'}).drop_duplicates('job_dirty')
    clean_nach = pd.merge(clean_nach, cleaning_jobs, how='left').drop(columns=['job_dirty', 'sum_curr', 'sum_prev'])
    print_df(clean_nach, post_path + months[i])
    #print nach here month.xlsx
    pay = pd.read_excel(dir_names[i] + '\\' + pay_name)
    pay_clean = pay.iloc[:, [2, 3, 6, 10, 13, 14, 15, 16]]
    pay_clean.columns = ['inn', 'snils', 'job_dirty', 'type', 'pre_name', 'nach_sum', 'kfo', 'nach_type']
    pay_clean = pd.merge(pay_clean, cleaning_jobs, how='left').drop(columns=['job_dirty'])
    print_df(pay_clean, post_path + months[i] + '_nach')
    #print pay here month_nach.xlsx
    kadr = pd.read_excel(dir_names[i] + '\\' + kadr_name)
    kadr_clean = kadr.iloc[:, [2, 3, 7, 8, 9, 12]]
    kadr_clean.columns = ['inn', 'snils', 'job_dirty', 'type', 'stv', 'status']
    kadr_clean = pd.merge(kadr_clean, cleaning_jobs, how='left').drop(columns=['job_dirty'])
    print_df(kadr_clean, post_path + months[i] + '_kadr')
    #print kadr here month_kadr.xlsx

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_18956\3622959342.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_nach['stv'] = clean_nach.apply(lambda row: to_double(row, 'stv'), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_18956\3622959342.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_nach['stv'] = clean_nach.apply(lambda row: to_double(row, 'stv'), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_18956\3622959342.py:5: SettingWithCopyWarning: 
A value is trying t